<a href="https://colab.research.google.com/github/ThesanR/TorontoPermitClassification/blob/main/PermitClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#2015 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/fbceeefc-a24d-43d0-8009-a1ec872550d0/download/Cleared%20Permits%202015.zip
!unzip -q "Cleared Permits 2015.zip"
#2016 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/b6f1907e-5a45-43f7-8c3c-56e4edc4f42f/download/Cleared%20Permits%202016.zip
!unzip -q "Cleared Permits 2016.zip"
#2017-2023 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/23de66da-ed1a-434a-a0fa-d85ec55f8f20/download/Cleared%20Building%20Permits%20since%202017.csv
#Active Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/108c2bd1-6945-46f6-af92-02f5658ee7f7/resource/dfce3b7b-4f17-4a9d-9155-5e390a5ffa97/download/building-permits-active-permits.csv

Classification of Permit applications from the City of Toronto between the years 2000 and 2023.

In [3]:
import pandas as pd
import numpy as np

df2015 = pd.read_csv('clearedpermits2015.csv', dtype={"POSTAL": str, "REVISION_NUM": str})
df2016 = pd.read_csv('clearedpermits2016.csv', dtype={"POSTAL": str, "REVISION_NUM": str})
df2017_2023 = pd.read_csv('Cleared Building Permits since 2017.csv', dtype={"POSTAL": str, "REVISION_NUM": str})
df = pd.concat([df2015, df2016, df2017_2023], ignore_index=True, sort=False)
df = df.drop(df[(df.POSTAL == "   ") | (df.EST_CONST_COST == "DO NOT UPDATE OR DELETE THIS INFO FIELD") | (df.STATUS != "Closed")].index)
df = df[['PERMIT_NUM', 'PERMIT_TYPE', 'STRUCTURE_TYPE', 'WORK', 'POSTAL', 'APPLICATION_DATE', 'ISSUED_DATE', 'EST_CONST_COST']]
df = df.dropna()
pd.set_option('display.max_rows', 25)

#df['EST_CONST_COST'] = ((df['EST_CONST_COST'].replace(",","", regex=True)).astype(float)).astype(int)
dftemp = ((df['EST_CONST_COST'].replace(",","", regex=True)).astype(float)).astype(int)
df = df.drop('EST_CONST_COST', axis=1)
df = pd.concat([df, dftemp], axis=1)
df = df.drop(df[df.EST_CONST_COST < 100].index)

df = df[~df['PERMIT_TYPE'].isin((df['PERMIT_TYPE'].value_counts())[(df['PERMIT_TYPE'].value_counts()) < 500].index)]
df = df[~df['WORK'].isin((df['WORK'].value_counts())[(df['WORK'].value_counts()) < 1000].index)]
df = df[~df['STRUCTURE_TYPE'].isin((df['STRUCTURE_TYPE'].value_counts())[(df['STRUCTURE_TYPE'].value_counts()) < 500].index)]
df = df[~df['PERMIT_NUM'].isin((df['PERMIT_NUM'].value_counts())[(df['PERMIT_NUM'].value_counts()) > 1].index)]

#print(df.PERMIT_NUM.value_counts(), "\n")
#print(df.POSTAL.value_counts(), "\n")
#print(df.PERMIT_TYPE.value_counts(), "\n")
#print(df.WORK.value_counts(),"\n")
#print(df.STRUCTURE_TYPE.value_counts(), "\n")

df

,PERMIT_NUM,PERMIT_TYPE,STRUCTURE_TYPE,WORK,POSTAL,APPLICATION_DATE,ISSUED_DATE,EST_CONST_COST
16,00 126897 CMB,Residential Building Permit,SFD - Semi-Detached,New Building,M6M,2000/02/10,2000/02/17,135000
27,00 130757 CMB,Residential Building Permit,SFD - Semi-Detached,New Building,M6P,2000/02/28,2002/07/15,125000
30,00 130904 CMB,Residential Building Permit,SFD - Semi-Detached,New Building,M6N,2000/02/28,2002/09/04,100000
33,00 130917 CMB,Residential Building Permit,SFD - Semi-Detached,New Building,M6N,2000/02/28,2002/09/04,100000
36,00 130919 CMB,Residential Building Permit,SFD - Semi-Detached,New Building,M6N,2000/02/28,2002/09/04,100000
...,...,...,...,...,...,...,...,...
339183,98 021551 BLD,Building Additions/Alterations,Restaurant Greater Than 30 Seats,Interior Alterations,M6J,2017-03-21,2017-05-10,50000
339564,99 101249 COM,Residential Building Permit,SFD - Detached,New Building,M5M,1999-10-06,1999-11-19,141000
339573,99 104374 COM,Residential Building Permit,Apartment Building,New Building,M6M,1999-10-21,1999-12-02,3000000
339605,99 114247 CMB,Residential Building Permit,SFD - Detached,New Building,M9A,1999-12-02,2001-05-10,450000
